In [1]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import whisper
import gradio as gr

C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def transcribe(audio_file):
    model = whisper.load_model('base')
    result = model.transcribe(audio_file)
    return result['text']

# LangChain

In [4]:
%pip install chromadb==0.4.15
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
import getpass
openai_api_key = getpass.getpass('Enter OpenAI API key:')

In [6]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [7]:
loader = CSVLoader(file_path='chunk_text_db.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
})
data = loader.load()

In [8]:
vectorstore = Chroma.from_documents(documents=data, 
                                    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [9]:
retriever = vectorstore.as_retriever()

## Retrieve Relevant Audio

In [10]:
def retrieve_audio_location(doc):
    split_doc = doc.rsplit(' ', 1)
    return 'C:/Users/natha/OneDrive/Desktop/coding projects/audio_search/audio_chunks/' + split_doc[1]

In [11]:
def retrieve_audio_chunk(query):
    search = transcribe(query)
    retrieved_docs = retriever.get_relevant_documents(
    search
    #query
    )
    best_fit = retrieved_docs[0].page_content
    audio_path = retrieve_audio_location(doc=best_fit)
    return audio_path

In [42]:
gr.Interface(fn=retrieve_audio_chunk, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs='audio').launch(debug=True)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Keyboard interruption in main thread... closing server.


## QA

In [39]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


TTS:

In [12]:
!pip install --upgrade --user transformers accelerate

     -------------------------------------- 126.8/126.8 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 8.2/8.2 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.0.11 requires pydantic<2.0, but you have pydantic 2.5.2 which is incompatible.


In [13]:
from transformers import VitsModel, AutoTokenizer
import torch

In [14]:
model = VitsModel.from_pretrained("facebook/mms-tts-eng")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.1.wavenet.res_skip_layers.3.weight_v', 'posterior_encoder.wavenet.in_layers.9.weight_v', 'posterior_encoder.wavenet.res_skip_layers.14.weight_v', 'posterior_encoder.wavenet.in_layers.4.weight_g', 'posterior_encoder.wavenet.res_skip_layers.8.weight_g', 'posterior_encoder.wavenet.res_skip_layers.12.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'posterior_encoder.wavenet.res_skip_layers.2.weight_g', 'flow.flows.1.wavenet.in_layers.2.weight_g', 'flow.flows.1.wavenet.res_skip_layers.3.weight_g', 'posterior_encoder.wavenet.res_skip_layers.15.weight_v', 'flow.flows.2.wavenet.in_layers.0.weight_v', 'flow.flows.3.wavenet.in_layers.0.weight_g', 'posterior_encoder.wavenet.in_layers.5.weight_g', 'flow.flows.3.wavenet.res_skip_layers.3.weight_v', 'posterior_encoder.wavenet.in_layers.8.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.r

In [47]:
text = "some example text in the English language"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

In [48]:
output

tensor([[-1.1510e-05, -4.0525e-05, -3.1603e-05,  ...,  4.4025e-05,
          2.8838e-05, -2.0677e-05]])

In [8]:
from IPython.display import Audio

Audio(output.numpy(), rate=model.config.sampling_rate)


In [52]:
import torch
from scipy.io.wavfile import write


# Reshape the tensor to have a shape of (number of channels, number of samples)
audio_data = output.reshape(-1).numpy()

# Scale the values to the appropriate range for audio data
scaled_audio_data = audio_data * 32767  # Assuming 16-bit signed integer format

# Convert the data type to 16-bit signed integer
scaled_audio_data = scaled_audio_data.astype('int16')

# Write the audio data to a WAV file
write('output_audio.wav', 16000, scaled_audio_data) 

# Gradio

In [36]:
import wave
import numpy as np

In [41]:
%pip show gradio whisper langchainhub chromadb==0.4.15

Name: gradio
Version: 4.8.0
Summary: Python library for easily interacting with trained machine learning models
Home-page: 
Author: 
Author-email: Abubakar Abid <team@gradio.app>, Ali Abid <team@gradio.app>, Ali Abdalla <team@gradio.app>, Dawood Khan <team@gradio.app>, Ahsen Khaliq <team@gradio.app>, Pete Allen <team@gradio.app>, Ömer Faruk Özdemir <team@gradio.app>, Freddy A Boulton <team@gradio.app>
License: 
Location: c:\users\natha\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: aiofiles, altair, fastapi, ffmpy, gradio-client, httpx, huggingface-hub, importlib-resources, jinja2, markupsafe, matplotlib, numpy, orjson, packaging, pandas, pillow, pydantic, pydub, python-multipart, pyyaml, requests, semantic-version, tomlkit, typer, typing-extensions, uvicorn
Required-by: 
---
Name: langchainhub
Version: 0.1.14
Summary: 
Home-page: 
Author: LangChain
Author-email: support@langchain.dev
License: 
Locat

In [53]:
def generate_qa_audio(query):
    question = transcribe(query)
    prompt = hub.pull("rlm/rag-prompt")
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    answer = rag_chain.invoke(question)
    inputs = tokenizer(answer, return_tensors="pt")

    with torch.no_grad():
        output = model(**inputs).waveform
        
        
    # Reshape the tensor to have a shape of (number of channels, number of samples)
    audio_data = output.reshape(-1).numpy()

    # Scale the values to the appropriate range for audio data
    scaled_audio_data = audio_data * 32767  # Assuming 16-bit signed integer format

    # Convert the data type to 16-bit signed integer
    scaled_audio_data = scaled_audio_data.astype('int16')

    # Write the audio data to a WAV file
    write('answer.wav', 16000, scaled_audio_data) 
        
    return 'C:/Users/natha/OneDrive/Desktop/coding projects/audio_search/answer.wav'

In [54]:
gr.Interface(fn=generate_qa_audio, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs=[
                'audio'
                ]).launch(debug=True)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\protocols\http\httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\fastapi\applications.py", line 1106, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\starlet

Keyboard interruption in main thread... closing server.


# Combine both apps

In [55]:
qa = gr.Interface(fn=generate_qa_audio, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs=[
                'audio'
                ],
            description="Ask a question and receive an answer from the biblical data.")

search = gr.Interface(fn=retrieve_audio_chunk, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs='audio',
            description="Find audio clips about a specific topic.")

In [56]:
demo = gr.TabbedInterface([search, qa], ["Search", "QA"])
demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\protocols\http\httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\fastapi\applications.py", line 1106, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\starlet